In [2]:
import streamlit as st
import pandas as pd
import altair as alt
from sqlalchemy import create_engine
from scripts.config import DB_CONNECTION_STRING

In [3]:
engine = create_engine(DB_CONNECTION_STRING)
city = "Tokyo"
sql = "SELECT * FROM weather_daily WHERE city = %s"
df_daily = pd.read_sql(sql=sql, con=engine, params=(city,))
df_daily.head()

,date,city,temperature_max,temperature_min,precipitation_sum,weather_code
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0,3.0
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0,3.0
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0,1.0
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6,73.0
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0,2.0


In [4]:
df_daily = df_daily.rename(columns={
    'temperature_max': 'temp_max',
    'temperature_min': 'temp_min',
    'precipitation_sum': 'precipitation'
})


In [5]:
df_daily['weather_code'].unique()

array([ 3.,  1., 73.,  2., 53., 63., 51., 55., 71., 65.,  0., 61., 75.])

In [6]:
# weather_map = {
#     0: "Clear sky",
#     1: "Mainly clear",
#     2: "Partly cloudy",
#     3: "Overcast",
#     51: "Light drizzle",
#     53: "Moderate drizzle",
#     55: "Dense drizzle",
#     61: "Slight rain",
#     63: "Moderate rain",
#     65: "Heavy rain",
#     71: "Slight snow fall",
#     73: "Moderate snow fall",
#     75: "Heavy snow fall"
# }

# Weather short name mapping
weather_desc_map = {
    0: "sun",
    1: "sun",
    2: "sun",
    3: "cloud",
    51: "drizzle",
    53: "drizzle",
    55: "drizzle",
    61: "rain",
    63: "rain",
    65: "rain",
    71: "snow",
    73: "snow",
    75: "snow"
}

# df_daily['weather_desc'] = df_daily['weather_code'].map(weather_map)
df_daily['weather_desc'] = df_daily['weather_code'].map(weather_desc_map)
df_daily.head()


,date,city,temp_max,temp_min,precipitation,weather_code,weather_desc
0,2020-01-01 00:00:00+00:00,Tokyo,8.25,0.00,0.0,3.0,cloud
1,2020-01-02 00:00:00+00:00,Tokyo,11.50,-1.20,0.0,3.0,cloud
2,2020-01-03 00:00:00+00:00,Tokyo,10.30,2.70,0.0,1.0,sun
3,2020-01-04 00:00:00+00:00,Tokyo,10.90,1.15,5.6,73.0,snow
4,2020-01-05 00:00:00+00:00,Tokyo,9.35,2.15,0.0,2.0,sun


In [7]:
df_daily['date'] = pd.to_datetime(df_daily['date'])
df_daily['year'] = df_daily['date'].dt.year
df_daily = df_daily[df_daily['year'] == 2025]
df_daily.head()

,date,city,temp_max,temp_min,precipitation,weather_code,weather_desc,year
1827,2025-01-01 00:00:00+00:00,Tokyo,11.15,1.75,0.0,0.0,sun,2025
1828,2025-01-02 00:00:00+00:00,Tokyo,13.05,2.50,0.1,51.0,drizzle,2025
1829,2025-01-03 00:00:00+00:00,Tokyo,6.20,0.60,2.1,51.0,drizzle,2025
1830,2025-01-04 00:00:00+00:00,Tokyo,8.20,-1.50,0.0,3.0,cloud,2025
1831,2025-01-05 00:00:00+00:00,Tokyo,8.60,0.30,0.0,3.0,cloud,2025


In [8]:
import plotly.express as px

fig = px.line(
    df_daily,
    x='date',
    y='temp_min',
    labels={
        'date': 'date',
        'temp_min': 'Temperature_min (°C)',
        'year': 'year'
    },
    title="Weather Tokyo 2025"
)
fig.update_layout(
    hovermode='x unified',
    xaxis=dict(
        tickformat="%b %d"
    )
)
fig.update_traces(line_color='red')
fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'date=%{x}<br>Temperature_min (°C)=%{y}<extra></extra>',
              'legendgroup': '',
              'line': {'color': 'red', 'dash': 'solid'},
              'marker': {'symbol': 'circle'},
              'mode': 'lines',
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'type': 'scatter',
              'x': array(['2025-01-01T00:00:00.000000000', '2025-01-02T00:00:00.000000000',
                          '2025-01-03T00:00:00.000000000', ..., '2025-12-29T00:00:00.000000000',
                          '2025-12-30T00:00:00.000000000', '2025-12-31T00:00:00.000000000'],
                         shape=(365,), dtype='datetime64[ns]'),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAA/D8AAAAAAAAEQDMzMzMzM+' ... 'ZmZu4/AAAAAAAAAkAzMzMzMzMHQA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'hovermode': 'x unified',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Weather Tokyo 2025'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'tickformat': '%b %d', 'title': {'text': 'date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'Temperature_min (°C)'}}}
})

In [16]:
weather_count = df_daily['weather_desc'].value_counts().reset_index()
weather_count.columns = ['weather', 'count']
weather_count


,weather,count
0,cloud,136
1,drizzle,108
2,rain,77
3,sun,36
4,snow,8


In [10]:
weather_colors = {      
    'cloud': '#95A5A6',    
    'drizzle': '#74B9FF',  
    'rain': '#0984E3',     
    'snow': '#DFE6E9',
    'sun': '#FDB813',   
}
fig = px.pie(
    weather_count,
    names='weather',
    values='count',
    title='Weather Distribution in Tokyo 2022',
    color_discrete_sequence=['#95A5A6', '#74B9FF', '#0984E3', '#FDB813', '#DFE6E9']
)
fig.update_traces(textinfo='label', textposition='auto')
fig


ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'domain': {'x': [0.0, 1.0], 'y': [0.0, 1.0]},
              'hovertemplate': 'weather=%{label}<br>count=%{value}<extra></extra>',
              'labels': array(['cloud', 'drizzle', 'rain', 'sun', 'snow'], dtype=object),
              'legendgroup': '',
              'name': '',
              'showlegend': True,
              'textinfo': 'label',
              'textposition': 'auto',
              'type': 'pie',
              'values': {'bdata': 'iABsAE0AJAAIAA==', 'dtype': 'i2'}}],
    'layout': {'legend': {'tracegroupgap': 0},
               'piecolorway': [#95A5A6, #74B9FF, #0984E3, #FDB813, #DFE6E9],
               'template': '...',
               'title': {'text': 'Weather Distribution in Tokyo 2022'}}
})

In [11]:
df_daily['month'] = df_daily['date'].dt.month_name()
df_daily.head()

,date,city,temp_max,temp_min,precipitation,weather_code,weather_desc,year,month
1827,2025-01-01 00:00:00+00:00,Tokyo,11.15,1.75,0.0,0.0,sun,2025,January
1828,2025-01-02 00:00:00+00:00,Tokyo,13.05,2.50,0.1,51.0,drizzle,2025,January
1829,2025-01-03 00:00:00+00:00,Tokyo,6.20,0.60,2.1,51.0,drizzle,2025,January
1830,2025-01-04 00:00:00+00:00,Tokyo,8.20,-1.50,0.0,3.0,cloud,2025,January
1831,2025-01-05 00:00:00+00:00,Tokyo,8.60,0.30,0.0,3.0,cloud,2025,January


In [12]:
df_percent = df_daily.groupby(['month', 'weather_desc']).size().reset_index(name='count')
df_percent['percent'] = (
    df_percent['count'] / df_percent.groupby('month')['count'].transform('sum')
)
df_percent['month'] = df_percent['month'].str[0:3]
df_percent.head()

,month,weather_desc,count,percent
0,Apr,cloud,10,0.333333
1,Apr,drizzle,8,0.266667
2,Apr,rain,10,0.333333
3,Apr,snow,1,0.033333
4,Apr,sun,1,0.033333


In [13]:
month_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
weather_colors = {
    'sun': '#FDB813',      
    'cloud': '#95A5A6',    
    'drizzle': '#74B9FF',  
    'rain': '#0984E3',     
    'snow': '#DFE6E9'      
}
fig = px.bar(
    df_percent,
    x='month',
    y='percent',
    color='weather_desc',
    title='Monthly weather breakdown',
    color_discrete_map=weather_colors,
    category_orders={'month': month_order}
)
fig.update_layout(
    barmode='stack',
    yaxis_tickformat='.0%',
    yaxis_title='days',           
    xaxis_title='month',          
    legend_title='weather',     
    height=500,                   
    plot_bgcolor='white', 
    showlegend=True
)
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'weather_desc=cloud<br>month=%{x}<br>percent=%{y}<extra></extra>',
              'legendgroup': 'cloud',
              'marker': {'color': '#95A5A6', 'pattern': {'shape': ''}},
              'name': 'cloud',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov',
                          'Oct', 'Sep'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('VVVVVVVV1T+llFJKKaXUP9daa6211t' ... 'IiIiIi4j+21lprrbXWPzMzMzMzM9M/'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'weather_desc=drizzle<br>month=%{x}<br>percent=%{y}<extra></extra>',
              'legendgroup': 'drizzle',
              'marker': {'color': '#74B9FF', 'pattern': {'shape': ''}},
              'name': 'drizzle',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov',
                          'Oct', 'Sep'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('ERERERER0T/nnHPOOefcP8YYY4wxxs' ... 'VVVVVVxT/GGGOMMcbYPxEREREREdE/'),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'weather_desc=rain<br>month=%{x}<br>percent=%{y}<extra></extra>',
              'legendgroup': 'rain',
              'marker': {'color': '#0984E3', 'pattern': {'shape': ''}},
              'name': 'rain',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Apr', 'Aug', 'Dec', 'Jan', 'Jul', 'Jun', 'Mar', 'May', 'Nov', 'Oct',
                          'Sep'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('VVVVVVVV1T+EEEIIIYTAP4QQQgghhM' ... 'mZmbk/hBBCCCGE0D+amZmZmZnZPw=='),
                    'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'weather_desc=snow<br>month=%{x}<br>percent=%{y}<extra></extra>',
              'legendgroup': 'snow',
              'marker': {'color': '#DFE6E9', 'pattern': {'shape': ''}},
              'name': 'snow',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Apr', 'Feb', 'Mar'], dtype=object),
              'xaxis': 'x',
              'y': {'bdata': 'ERERERERoT+SJEmSJEmiP8YYY4wxxsg/', 'dtype': 'f8'},
              'yaxis': 'y'},
             {'hovertemplate': 'weather_desc=sun<br>month=%{x}<br>percent=%{y}<extra></extra>',
              'legendgroup': 'sun',
              'marker': {'color': '#FDB813', 'pattern': {'shape': ''}},
              'name': 'sun',
              'orientation': 'v',
              'showlegend': True,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['Apr', 'Aug', 'Dec', 'Feb', 'Jan', 'Jul', 'Jun', 'Mar', 'Nov', 'Sep'],
                         dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('ERERERERoT/GGGOMMca4P4QQQgghhN' ... 'IIIYSwP1VVVVVVVcU/ERERERERoT8='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'stack',
               'height': 500,
               'legend': {'title': {'text': 'weather'}, 'tracegroupgap': 0},
               'plot_bgcolor': 'white',
               'showlegend': True,
               'template': '...',
               'title': {'text': 'Monthly weather breakdown'},
               'xaxis': {'anchor': 'y',
                         'categoryarray': [Jan, Feb, Mar, Apr, May, Jun, Jul, Aug,
                                           Sep, Oct, Nov, Dec],
                         'categoryorder': 'array',
       

In [14]:
fig = px.bar(
    df_daily,
    x='month',
    y='precipitation',
    labels={
        'month': 'date',
        'precipitation': 'precipitation (mm)'
    },
    title='Precipitation'
)
fig.update_traces(marker_color='lightblue')
fig

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'hovertemplate': 'date=%{x}<br>precipitation (mm)=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': 'lightblue', 'pattern': {'shape': ''}},
              'name': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['January', 'January', 'January', ..., 'December', 'December',
                          'December'], shape=(365,), dtype=object),
              'xaxis': 'x',
              'y': {'bdata': ('AAAAAAAAAACamZmZmZm5P83MzMzMzA' ... 'AAAAAAAAAAAAAAAAAAAAAAAAAAAA=='),
                    'dtype': 'f8'},
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'Precipitation'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': 'date'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': 'precipitation (mm)'}}}
})